# Usando como exemplo a Syngenta - sementes NK

## Importando as bibliotecas necessárias:


In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests

# Definindo funções:

In [238]:
def abre_e_scrap(url):
    
    """Esta função abre o url dado como parâmetro e retorna a 'sopa de html'.
    Parâmetro: url
    Retorno: uma sopa de html da página."""
    
    html = urlopen(url)
    scrap = BeautifulSoup(html, 'html.parser')
    return scrap

In [228]:
#teste da função abre_e_scrap()

url = 'http://www.portalsyngenta.com.br/sementes/nk-soja'
scrap = abre_e_scrap(url)

### Explorado a página html com 6 cards de mesma estrutura: < div class = card-text-2
#### Criando uma função que busca com `find_all` e retorna uma lista com os links de cada card

In [229]:
# Buscando no html os cards repetidos, com <div class: "class_card"

def find_links_in_cards(scrap, class_card):
    
    """ Função que recebe a sopa e a string dos cards, 
    passa por todos esses cards e retorna os links contidos neles.
    Parâmetros: 
    * scrap: página html obtida através do BeautifulSoup
    * class_card: qual a str correspondente ao card a ser encontrado
    Retorno: uma lista contendo os links contidos em cada card."""
    
    cards = scrap.find_all('div', {'class': class_card})
    
    links = []

    for link in cards:
        link = link.a['href']
        links.append(link)
    
    return links

In [234]:
# testando a função find_links_in_cards

class_card = "card-text-2 card-portfolio"
links = find_links_in_cards(scrap, class_card)
links[:2]

['/sementes/nk-soja/nk-8770-ipro', '/sementes/nk-soja/nk-7777-ipro']

## Dependendo do link obtido, vai ser necessário complementá-lo:

In [235]:
def complement_link(lista_links, link_principal):
    
    """Função que recebe uma lista com links parciais e complementa-os usando o link principal.
    Parâmetros:
    * lista_links: lista contendo links parciais
    * link_principal: início do link que foi oculto nos links parciais
    Retorno: uma lista com os links completos"""
    
    links_completos = []
    
    for link in lista_links:
        link = link_principal+link
        links_completos.append(link)
        
    return links_completos

In [237]:
# testando a função complement_link:

principal = 'http://www.portalsyngenta.com.br'
links_completos = complement_link(links, principal)
links_completos[:2]

['http://www.portalsyngenta.com.br/sementes/nk-soja/nk-8770-ipro',
 'http://www.portalsyngenta.com.br/sementes/nk-soja/nk-7777-ipro']

## Explorando cada link e coletando as informações de mesma estrutura html:              
* Elemento: < p > ou < div >
* Classe: < elemento **class** : " string "

In [242]:
def find_similar_info(lista_links, elemento, class_busca, tamanho_nome):

    """Função que varre uma lista com links e retorna os resultados iguais agregados de casa site 
    Parâmetros: 
    * scrap: 'sopa' do html obtido com o Beautiful Soup
    * elemento: passando "p" sendo <p> ou "div" sendo <div>, entre outros.
    * class_busca: passando uma string, como "card-texto", que fica após class= "card-texto"
    * tamanho_nome: últimas letras contidas no link. 
        Por exemplo, passando um número: ( -13 ) --> a str 'tmg-2375-ipro' tem 13 letras.
        Site exemplo: https://www.tmg.agr.br/ptbr/cultivar/tmg-2375-ipro"""


    lista_busca = []

    for link in lista_links:

        scrap = abre_e_scrap(link)        # criando a sopa de html de cada página

        busca = scrap.find_all(elemento, {'class': class_busca})

        for item in busca:
            lista_busca.append(link[tamanho_nome:])   # nome das sementes
            lista_busca.append(item.p.text)
    
    return lista_busca

In [244]:
# testando a função find_similar_info:

caracteristicas = find_similar_info(links_completos, 'div', "prop-nk-titulo", -12)
caracteristicas[:7]

['nk-8770-ipro',
 'Hábito de Crescimento',
 'nk-8770-ipro',
 'Cor da Flor',
 'nk-8770-ipro',
 'Grupo de Maturação',
 'nk-8770-ipro']

In [245]:
# testando a função find_simialr_info para coletar a descrição das características:

descricao = find_similar_info(links_completos, 'div', 'prop-nk-descricao', -12)
descricao[:7]

['nk-8770-ipro',
 'semi-determinado',
 'nk-8770-ipro',
 'ROXA',
 'nk-8770-ipro',
 '8.7',
 'nk-8770-ipro']

## Criando um array e dando um reshape:

In [254]:
def criando_df(lista, numero_colunas, lista_header):
    
    """Função que recebe uma lista contendo nome e informação, alternados, e cria um dataframe.
    Parâmetros: 
    * lista: lista a ser usada como base para o DataFrame
    * numero_colunas: quantas colunas vão ser feitas a partir dos dados da lista
    * lista_header: uma lista contendo o nome das colunas do DataFrame
    Retorno: DataFrame"""
    
    numero_linhas = len(lista)/numero_colunas
    numero_linhas = int(numero_linhas)
    numero_colunas = int(numero_colunas)
    array = np.array(lista).reshape(numero_linhas, numero_colunas)

    df = pd.DataFrame(data=array, columns=lista_header)
    
    return df

In [257]:
# testando a função criando_df com as caracteristicas

df_caract = criando_df(caracteristicas, 2, ['Nome', 'caracteristica'])
df_caract.head()

,Nome,caracteristica
0,nk-8770-ipro,Hábito de Crescimento
1,nk-8770-ipro,Cor da Flor
2,nk-8770-ipro,Grupo de Maturação
3,nk-8770-ipro,Pubescência
4,nk-8770-ipro,Acamamento


In [287]:
# testando a função criando_df com a descrição

df_descricao = criando_df(descricao, 2, ['Nome', 'Descrição'])
df_descricao.head()

,Nome,Descrição
0,nk-8770-ipro,semi-determinado
1,nk-8770-ipro,ROXA
2,nk-8770-ipro,8.7
3,nk-8770-ipro,CINZA
4,nk-8770-ipro,Tolerante


In [ ]:
df_descricao = df_descricao.drop('Nome', axis=1)

In [300]:
# juntando os DataFrames com as características e a descrição de cada semente:

df_caract = df_caract.join(df_descricao)
df_caract.head()

,Nome,caracteristica,Descrição
0,nk-8770-ipro,Hábito de Crescimento,semi-determinado
1,nk-8770-ipro,Cor da Flor,ROXA
2,nk-8770-ipro,Grupo de Maturação,8.7
3,nk-8770-ipro,Pubescência,CINZA
4,nk-8770-ipro,Acamamento,Tolerante


## 7. Criando DataFrame das caracteristicas + nomes das colunas
* a lista_titulos se repente de 9 em 9 elementos
* pegando os elementos únicos e adicionando 'Nome'
* colocando como header a lista contendo 10 elementos com os nomes das colunas

In [222]:
lista_titulos = lista_titulos[:9].copy()
lista_titulos.append('Nome')
df_caract.columns = lista_titulos
df_caract.head(2)

,Hábito de Crescimento,Cor da Flor,Grupo de Maturação,Pubescência,Acamamento,Hilo,Altura da Planta (cm),"PMS Médio 5,5 mm",Inserção de Vagem (cm),Nome
0,semi-determinado,ROXA,8.7,CINZA,Tolerante,Marrom Claro,97 - 112,140 - 190 KG,14 - 19,nk-8770-ipro
1,INDETERMINADO,ROXA,7.7,CINZA,Tolerante,Marrom Claro,80 - 95,190 G,15 - 20,nk-7777-ipro


## mesmo processo de criação de DataFrame para:
* ### Benefícios
* ### Recomendações

In [239]:
array_benef = np.array(lista_beneficios)
array_benef = array_benef.reshape(21,2)
df_benef = pd.DataFrame(array_benef)

df_benef.columns = ['Nome', 'Beneficios']
df_benef['Subtitulo'] = lista_benef_subtitulo[:21]

array_benef

array([['nk-8770-ipro', 'Execução'],
       ['nk-8770-ipro', 'Performance'],
       ['nk-8770-ipro', 'Plantio'],
       ['nk-7777-ipro', 'Potencial'],
       ['nk-7777-ipro', 'Adaptação'],
       ['nk-7777-ipro', 'Plantio'],
       ['nk-7777-ipro', 'Manejo'],
       ['nk-8448-ipro', 'Potencial'],
       ['nk-8448-ipro', 'Adaptação'],
       ['nk-8448-ipro', 'Plantio'],
       ['nk-8448-ipro', 'Manejo'],
       ['nk-8301-ipro', 'Segurança'],
       ['nk-8301-ipro', 'Manejo'],
       ['nk-7201-ipro', 'Plantio'],
       ['nk-7201-ipro', 'Desempenho'],
       ['nk-7201-ipro', 'Segurança'],
       ['nk-6201-ipro', 'Estrutura'],
       ['nk-6201-ipro', 'Bom desempenho'],
       ['nk-6201-ipro', 'Boa tolerância'],
       ['nk-6201-ipro', 'Segurança'],
       ['nk-6201-ipro', 'Plantio']], dtype='<U14')

## 8. Lendo as tabelas dos sites (html)
* ### Recomendação Técnica

In [274]:
url = 'http://www.portalsyngenta.com.br/sementes/nk-soja/nk-6201-ipro'
class_table = 'table-sementes'   # o que será procurado na página

table_recom = pd.read_html(url)

html = urlopen(url)
scrap = BeautifulSoup(html, 'html.parser')

table_recom_html = scrap.find('table', {'class': class_table})

lista_temp = []         # lista temporária, antes de tratar as cores
lista_cel_recom = []      # lista pós tratamento, recebe apenas os valores da legenda

for row in table_recom_html:
    for i in row:
        i = str(i)
        if "td-data-gray" in i:
            i = i.replace('"td-data-gray"', '"Não recomendável"')
            i = i.replace('"td-data-gray border-right"', '"Não recomendável"')
            lista_temp.append(i)
        elif 'td-data-light-blue' in i:
            i = i.replace('"td-data-light-blue"', '"Aceitável"')
            i = i.replace('"td-data-light-blue border-right"', '"Aceitável"')
            lista_temp.append(i)
        elif 'td-data-dark-blue' in i:
            i = i.replace('"td-data-dark-blue"', '"Recomendado"')
            i = i.replace('"td-data-dark-blue border-right"', '"Recomendado"')
            lista_temp.append(i)
            
for i in lista_temp:
    i = i.replace('<td class=', '')
    i = i.replace('></td>', '')
    lista_cel_recom.append(i)

In [275]:
df_recom = pd.DataFrame(table_recom[0])
df_recom

,Unnamed: 0,Unnamed: 1,Setembro,Setembro.1,Setembro.2,Outubro,Outubro.1,Outubro.2,Novembro,Novembro.1,Novembro.2,Dezembro,Dezembro.1,Dezembro.2,Unnamed: 14
0,Região,Ciclo,1.0,2.0,3.0,1.0,2.0,3.0,1.0,2.0,3.0,1.0,2.0,3.0,POP.(mil plts/ha)L
1,201W,129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,260-280
2,201W,129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,260-300
3,201N,112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,260-280
4,201N,112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,260-300
5,202,115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,260-280
6,202,115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,260-320
7,202,115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,280-320


In [ ]:
array_legendas_recom = np.array(lista_cel_recom).reshape(7, 12)


In [284]:
df_legendas = pd.DataFrame(array_legendas_recom)
df_legendas.columns = ['Setembro', 'Setembro.1', 'Setembro.2',
       'Outubro', 'Outubro.1', 'Outubro.2', 'Novembro', 'Novembro.1',
       'Novembro.2', 'Dezembro', 'Dezembro.1', 'Dezembro.2']

df_legendas.index = range(1, 8)
df_legendas

,Setembro,Setembro.1,Setembro.2,Outubro,Outubro.1,Outubro.2,Novembro,Novembro.1,Novembro.2,Dezembro,Dezembro.1,Dezembro.2
1,"""Não recomendável""","""Não recomendável""","""Não recomendável""","""Recomendado""","""Recomendado""","""Recomendado""","""Recomendado""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Não recomendável"""
2,"""Não recomendável""","""Aceitável""","""Recomendado""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Aceitável""","""Aceitável""","""Não recomendável""","""Não recomendável""","""Não recomendável"""
3,"""Não recomendável""","""Não recomendável""","""Não recomendável""","""Recomendado""","""Recomendado""","""Recomendado""","""Recomendado""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Não recomendável"""
4,"""Não recomendável""","""Aceitável""","""Recomendado""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Aceitável""","""Aceitável""","""Não recomendável""","""Não recomendável""","""Não recomendável"""
5,"""Não recomendável""","""Não recomendável""","""Não recomendável""","""Recomendado""","""Recomendado""","""Recomendado""","""Recomendado""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Não recomendável"""
6,"""Não recomendável""","""Aceitável""","""Recomendado""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Aceitável""","""Aceitável""","""Não recomendável""","""Não recomendável""","""Não recomendável"""
7,"""Não recomendável""","""Aceitável""","""Recomendado""","""Recomendado""","""Recomendado""","""Recomendado""","""Aceitável""","""Aceitável""","""Aceitável""","""Não recomendável""","""Não recomendável""","""Não recomendável"""


In [285]:
df_recom.fillna(df_legendas)

,Unnamed: 0,Unnamed: 1,Setembro,Setembro.1,Setembro.2,Outubro,Outubro.1,Outubro.2,Novembro,Novembro.1,Novembro.2,Dezembro,Dezembro.1,Dezembro.2,Unnamed: 14
0,Região,Ciclo,1,2,3,1,2,3,1,2,3,1,2,3,POP.(mil plts/ha)L
1,201W,129,"""Não recomendável""","""Não recomendável""","""Não recomendável""","""Recomendado""","""Recomendado""","""Recomendado""","""Recomendado""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Não recomendável""",260-280
2,201W,129,"""Não recomendável""","""Aceitável""","""Recomendado""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Aceitável""","""Aceitável""","""Não recomendável""","""Não recomendável""","""Não recomendável""",260-300
3,201N,112,"""Não recomendável""","""Não recomendável""","""Não recomendável""","""Recomendado""","""Recomendado""","""Recomendado""","""Recomendado""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Não recomendável""",260-280
4,201N,112,"""Não recomendável""","""Aceitável""","""Recomendado""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Aceitável""","""Aceitável""","""Não recomendável""","""Não recomendável""","""Não recomendável""",260-300
5,202,115,"""Não recomendável""","""Não recomendável""","""Não recomendável""","""Recomendado""","""Recomendado""","""Recomendado""","""Recomendado""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Não recomendável""",260-280
6,202,115,"""Não recomendável""","""Aceitável""","""Recomendado""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Não recomendável""","""Aceitável""","""Aceitável""","""Não recomendável""","""Não recomendável""","""Não recomendável""",260-320
7,202,115,"""Não recomendável""","""Aceitável""","""Recomendado""","""Recomendado""","""Recomendado""","""Recomendado""","""Aceitável""","""Aceitável""","""Aceitável""","""Não recomendável""","""Não recomendável""","""Não recomendável""",280-320


## 9. Exportando para arquivo excel, dentro da pasta: planilhas/

In [82]:
df_caract.to_excel('planilhas/Syngenta_NK/caracteristicas.xlsx')
df_benef.to_excel('planilhas/Syngenta_NK/beneficios.xlsx')